In [ ]:
import os
import pandas as pd
from collections import defaultdict

# processed_data 폴더 경로
folder_path = "./filtered_data" ### 이게 filterd 데이터에서 가져오는게 맞아? 근데 processed 데이터로 바꾸면 밑에 어떤 파일에는 없지만, 다른 파일에는 포함된 날짜 개수가 몇 백개 생김

# 모든 파일에서 포함된 날짜 저장
date_counts = defaultdict(int)
file_dates = {}

# 파일 불러오기
for file in os.listdir(folder_path):
    if file.endswith(".csv") or file.endswith(".xlsx"):  # CSV 또는 Excel 파일만 처리
        file_path = os.path.join(folder_path, file)
        
        # 파일 형식에 따라 로드
        if file.endswith(".csv"):
            df = pd.read_csv(file_path, parse_dates=["Date"])
        else:
            df = pd.read_excel(file_path, parse_dates=["Date"])
        
        # 날짜 열이 존재하는지 확인
        if "Date" in df.columns:
            unique_dates = set(df["Date"].dt.date)
            file_dates[file] = unique_dates
            for date in unique_dates:
                date_counts[date] += 1

# 모든 파일에 포함된 날짜 개수
total_files = len(file_dates)
included_dates = sum(1 for count in date_counts.values() if count == total_files)
excluded_dates = sum(1 for count in date_counts.values() if count != total_files)

print(f"모든 파일에 포함된 날짜 개수: {included_dates}")
print(f"어떤 파일에는 없지만, 다른 파일에는 포함된 날짜 개수: {excluded_dates}")


모든 파일에 포함된 날짜 개수: 4471
어떤 파일에는 없지만, 다른 파일에는 포함된 날짜 개수: 0


In [12]:
# 누락된 날짜 확인 및 출력
missing_dates = {date for date, count in date_counts.items() if count != total_files}
missing_dates_per_file = {file: missing_dates - dates for file, dates in file_dates.items()}

print("어떤 파일에는 없지만, 다른 파일에는 포함된 날짜 목록:")
for file, missing in missing_dates_per_file.items():
    if missing:
        print(f"파일: {file}, 누락된 날짜: {sorted(missing)}")

어떤 파일에는 없지만, 다른 파일에는 포함된 날짜 목록:


In [13]:
import datetime
import holidays

# 미국 공휴일 설정
us_holidays = holidays.US()

# 주말 및 공휴일 확인 함수
def is_weekend_or_holiday(date):
    return date.weekday() >= 5 or date in us_holidays  # 주말(토:5, 일:6) 또는 공휴일

# 주말 또는 공휴일과 그렇지 않은 날짜 개수 세기
weekend_or_holiday_count = sum(1 for date in missing_dates if is_weekend_or_holiday(date))
non_weekend_or_holiday_count = len(missing_dates) - weekend_or_holiday_count

print(f"주말 또는 공휴일인 누락 날짜 개수: {weekend_or_holiday_count}")
print(f"주말 또는 공휴일이 아닌 누락 날짜 개수: {non_weekend_or_holiday_count}")


주말 또는 공휴일인 누락 날짜 개수: 0
주말 또는 공휴일이 아닌 누락 날짜 개수: 0


In [14]:
# 주말 또는 공휴일이 아닌 누락 날짜 출력
non_weekend_or_holiday_dates = [date for date in missing_dates if not is_weekend_or_holiday(date)]
print("주말 또는 공휴일이 아닌 누락 날짜:")
print(sorted(non_weekend_or_holiday_dates))

주말 또는 공휴일이 아닌 누락 날짜:
[]


In [15]:
import os
import pandas as pd

# 원본 데이터 폴더 경로 및 저장할 폴더 경로 설정
input_folder = "./data"
output_folder = "./filtered_data"
os.makedirs(output_folder, exist_ok=True)

# 모든 파일에서 포함된 날짜 저장
date_counts = {}
file_dates = {}

# 파일 불러오기
for file in os.listdir(input_folder):
    if file.endswith(".csv") or file.endswith(".xlsx"):  # CSV 또는 Excel 파일만 처리
        file_path = os.path.join(input_folder, file)
        
        # 파일 형식에 따라 로드
        if file.endswith(".csv"):
            df = pd.read_csv(file_path, parse_dates=["Date"])
        else:
            df = pd.read_excel(file_path, parse_dates=["Date"])
        
        # 날짜 열이 존재하는지 확인
        if "Date" in df.columns:
            unique_dates = set(df["Date"].dt.date)
            file_dates[file] = unique_dates
            for date in unique_dates:
                date_counts[date] = date_counts.get(date, 0) + 1

# 모든 파일에 포함된 날짜 식별
total_files = len(file_dates)
included_dates = {date for date, count in date_counts.items() if count == total_files}

# 모든 파일에서 포함되지 않은 날짜 제거 후 저장
for file, dates in file_dates.items():
    file_path = os.path.join(input_folder, file)
    
    # 파일 로드
    if file.endswith(".csv"):
        df = pd.read_csv(file_path, parse_dates=["Date"])
    else:
        df = pd.read_excel(file_path, parse_dates=["Date"])
    
    # 날짜 필터링
    df_filtered = df[df["Date"].dt.date.isin(included_dates)]
    
    # 필터링된 파일 저장
    output_path = os.path.join(output_folder, file)
    if file.endswith(".csv"):
        df_filtered.to_csv(output_path, index=False)
    else:
        df_filtered.to_excel(output_path, index=False)

print(f"필터링된 데이터가 '{output_folder}' 폴더에 저장되었습니다.")


필터링된 데이터가 './filtered_data' 폴더에 저장되었습니다.


In [16]:
output_folder = "./filtered_data"
os.makedirs(output_folder, exist_ok=True)

# 제외할 파일 목록
exclude_files = {"CBOE Volatility Index Historical Data.csv", "USD_KRW Historical Data.csv"}

# NaN 값 확인 (제외된 파일을 제외하고 검사)
print("\n파일별 NaN 값 확인 결과:")
for file in os.listdir(output_folder):
    if file in exclude_files:
        continue
    
    if file.endswith(".csv") or file.endswith(".xlsx"):
        file_path = os.path.join(output_folder, file)
        
        # 파일 로드
        if file.endswith(".csv"):
            df = pd.read_csv(file_path)
        else:
            df = pd.read_excel(file_path)
        
        # NaN 값 개수 확인
        nan_count = df.isna().sum().sum()
        if nan_count > 0:
            print(f"파일: {file} - NaN 값 개수: {nan_count}")
        else:
            print(f"파일: {file} - NaN 값 없음")



파일별 NaN 값 확인 결과:
파일: Crude Oil WTI Futures Historical Data.csv - NaN 값 개수: 1
파일: DIA ETF Stock Price History.csv - NaN 값 없음
파일: Gold Futures Historical Data.csv - NaN 값 개수: 1
파일: QQQ ETF Stock Price History.csv - NaN 값 없음
파일: SPY ETF Stock Price History.csv - NaN 값 없음
파일: United States 10-Year Bond Yield Historical Data.csv - NaN 값 없음


In [ ]:
import os
import pandas as pd

# 원본 데이터 폴더 경로 및 저장할 폴더 경로 설정
input_folder = "./data"
output_folder = "./filtered_data"
os.makedirs(output_folder, exist_ok=True)

# 제외할 파일 목록 (VIX 및 환율 데이터는 NaN 검사에서 제외)
exclude_files = {"CBOE Volatility Index Historical Data.csv", "USD_KRW Historical Data.csv"}

# NaN 값이 포함된 날짜 찾기 (VIX 및 환율 데이터 제외하고 검사)
nan_dates = set()
for file in os.listdir(output_folder):
    if file in exclude_files:
        continue
    
    if file.endswith(".csv") or file.endswith(".xlsx"):
        file_path = os.path.join(output_folder, file)
        
        # 파일 로드
        if file.endswith(".csv"):
            df = pd.read_csv(file_path, parse_dates=["Date"])
        else:
            df = pd.read_excel(file_path, parse_dates=["Date"])
        
        # NaN이 있는 날짜 찾기
        nan_rows = df[df.isna().any(axis=1)]["Date"].dt.date
        nan_dates.update(nan_rows)

# 모든 파일에서 NaN이 있는 날짜 제거 (VIX 및 환율 데이터 포함하여 처리)
for file in os.listdir(output_folder):
    if file.endswith(".csv") or file.endswith(".xlsx"):
        file_path = os.path.join(output_folder, file)
        
        # 파일 로드
        if file.endswith(".csv"):
            df = pd.read_csv(file_path, parse_dates=["Date"])
        else:
            df = pd.read_excel(file_path, parse_dates=["Date"])
        
        # NaN이 포함된 날짜 제거
        df_filtered = df[~df["Date"].dt.date.isin(nan_dates)]
        
        # 필터링된 파일 저장
        if file.endswith(".csv"):
            df_filtered.to_csv(file_path, index=False)
        else:
            df_filtered.to_excel(file_path, index=False)

print(f"모든 파일에서 NaN이 포함된 날짜를 제거하고 저장 완료.")


모든 파일에서 NaN이 포함된 날짜를 제거하고 저장 완료.


: 